In [47]:
import pandas as pd
import numpy as np
import wrangle as w

# modeling methods
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, f_regression, RFE

In [2]:
train, validate, test = w.wrangle_zillow()

opening data from local file


In [3]:
train_scaled, validate_scaled, test_scaled = w.scale_data(train, validate, test)

In [25]:
drops = ['tax_value','lot_size_binned', 'cars_garage']

In [26]:
X_train_scaled = train_scaled.drop(columns=drops)
y_train = train_scaled.tax_value
X_validate_scaled = validate_scaled.drop(columns=drops)
y_validate = validate_scaled.tax_value
X_test_scaled = test_scaled.drop(columns=drops)
y_test = test_scaled.tax_value

# Feature selection

In [39]:
# parameters: f_regression stats test, give me 3 features
f_selector = SelectKBest(f_regression, k=4)

# find the top 3 X's correlated with y
f_selector.fit(X_train_scaled, y_train)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature_kbest = X_train_scaled.iloc[:,feature_mask].columns.tolist()

In [40]:
f_feature_kbest

['bathrooms', 'area', 'garage_sqft', 'year_built']

In [41]:
lm = LinearRegression()
# parameters: f_regression stats test, give me 3 features
f_selector = RFE(lm, n_features_to_select=4)

# find the top 3 X's correlated with y
f_selector.fit(X_train_scaled, y_train)

# boolean mask of whether the column was selected or not. 
feature_mask = f_selector.get_support()

# get list of top K features. 
f_feature_rfe = X_train_scaled.iloc[:,feature_mask].columns.tolist()

In [42]:
f_feature_rfe

['bedrooms', 'bathrooms', 'area', 'garage_sqft']

# Establish Baseline

In [43]:
y_train = pd.DataFrame(y_train)
y_validate = pd.DataFrame(y_validate)

In [44]:
y_train['tax_value_pred_mean'] = y_train.tax_value.mean()
y_validate['tax_value_pred_mean'] = y_train.tax_value.mean()

y_train['tax_value_pred_med'] = y_train.tax_value.median()
y_validate['tax_value_pred_med'] = y_train.tax_value.median()

In [45]:
rmse_train_mu = mean_squared_error(y_train.tax_value,
                                   y_train.tax_value_pred_mean) ** .5
rmse_validate_mu = mean_squared_error(y_validate.tax_value, 
                                      y_validate.tax_value_pred_mean) ** (0.5)
rmse_train_med = mean_squared_error(y_train.tax_value, 
                                    y_train.tax_value_pred_med) ** .5
rmse_validate_med = mean_squared_error(y_validate.tax_value, 
                                       y_validate.tax_value_pred_med) ** (0.5)

In [48]:
metric_df = pd.DataFrame(data=[
    {
        'model' : 'mean_baseline',
        'RMSE_train' : rmse_train_mu,
        'RMSE_validate' : rmse_validate_mu,
        'difference' : rmse_validate_mu - rmse_train_mu,
        'R2' : explained_variance_score(y_validate.tax_value,
                                       y_validate.tax_value_pred_mean)
    }
])

In [49]:
metric_df

,model,RMSE_train,RMSE_validate,difference,R2
0,mean_baseline,243722.129324,246130.971338,2408.842014,0.0


In [50]:
# make la thing
lm = LinearRegression()
# fit za thing
lm.fit(X_train_scaled[f_feature], y_train.tax_value)
# usage of a thing
y_train['tax_value_pred_lm'] = lm.predict(X_train_scaled[f_feature])
# Evaluate: RMSE
rmse_train = mean_squared_error(y_train.tax_value, y_train.tax_value_pred_lm) ** .5

# repeat usage on validate
y_validate['tax_value_pred_lm'] = lm.predict(X_validate[f_feature])
# evaluate: RMSE
rmse_validate = mean_squared_error(y_validate.tax_value, y_validate.tax_value_pred_lm) ** .5

NameError: name 'X_validate' is not defined